# Train a Hypergraph Neural Network

In this notebook, we will create and train a two-step message passing network **HyperGAT** ([Ding et al., 2020](https://aclanthology.org/2020.emnlp-main.399.pdf)) in the hypergraph domain. We will use a benchmark dataset, shrec16, a collection of 3D meshes, to train the model to perform classification at the level of the hypergraph. 

Given a hypergraph $G=(\mathcal{V}, \mathcal{E})$, where $|\mathcal{V}|=n, |\mathcal{V}|=m$, let $X \in \mathbb{R}^{n \times d}$ and $Z \in \mathbb{R}^{m \times d'}$ denote the hidden node and hyperedge representations, respectively. 
    